<a href="https://colab.research.google.com/github/casset-org/app/blob/main/io.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import networkx as nx
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import statsmodels.api as sm
from scipy import stats
from google.colab import drive
import requests
import io

# --- Constantes e Configurações ---
DRIVE_MOUNT_PATH = "/content/drive"
GRAPHPATH = os.path.join(DRIVE_MOUNT_PATH, "MyDrive", "graficos")
SUMMARYPATH = os.path.join(DRIVE_MOUNT_PATH, "MyDrive", "sumarios")
DEFAULT_PALETTE = "Spectral"
FIGSIZE = (10, 6)
DATASET_URL = "https://docs.google.com/spreadsheets/d/1CukfW6MJBt0HoLYikhP46GsnzZva5K_ynKwPlAkVwQw/export?format=csv"

# Configuração de estilo
sns.set(style="darkgrid")
plt.rcParams.update({
    'figure.facecolor': '#282c34',
    'axes.facecolor': '#282c34',
    'axes.edgecolor': 'white',
    'axes.labelcolor': 'lime',
    'xtick.color': 'white',
    'ytick.color': 'white',
    'text.color': 'white',
    'grid.color': 'gray',
    'grid.linestyle': '--',
    'legend.facecolor': '#282c34',
    'legend.edgecolor': 'white',
    'figure.titlesize': 18,
    'axes.titlesize': 14,
    'axes.labelsize': 12,
})

# --- Funções Auxiliares ---

def save_fig(filename):
    filepath = os.path.join(GRAPHPATH, filename)
    os.makedirs(os.path.dirname(filepath), exist_ok=True)
    plt.savefig(filepath, dpi=300, bbox_inches='tight')
    plt.close()

def create_figure():
    return plt.figure(figsize=FIGSIZE)

def load_data_from_url(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        csv_content = response.content.decode('utf-8')
        df = pd.read_csv(io.StringIO(csv_content))
        return df
    except Exception as e:
        print(f"Erro ao carregar dados: {e}")
        return None

def generate_synthetic_data_improved(n_projects=50):
    np.random.seed(42)
    data = {
        'ProjetoID': range(1, n_projects + 1),
        'UsaCasset': np.random.choice([True, False], n_projects),
        'TempoEtapa1': np.random.randint(5, 15, n_projects),
        'TempoEtapa2': np.random.randint(7, 20, n_projects),
        'TempoEtapa3': np.random.randint(3, 10, n_projects),
        'InteracoesDocs': np.random.randint(10, 100, n_projects),
        'InteracoesEmail': np.random.randint(5, 50, n_projects),
        'InteracoesReunioes': np.random.randint(2, 15, n_projects),
        'QualidadeProduto': np.random.normal(7, 1.5, n_projects),  # Mantém como float
        'SatisfacaoParticipantes': np.random.normal(8, 1, n_projects),  # Mantém como float
        'NumParticipantes': np.random.randint(3, 8, n_projects),
        'CentralidadeComunicacao': np.random.uniform(0.3, 0.8, n_projects),
        'DiversidadeEquipe': np.random.uniform(0.2, 1, n_projects).round(2),
        'FrequenciaFeedback': np.random.randint(1, 6, n_projects),
        'LiderancaDistribuida': np.random.uniform(0.1, 1, n_projects).round(2),
        'ComplexidadeTarefa' : np.random.randint(1, 11, n_projects)
    }
    df = pd.DataFrame(data)

    # Efeito Base do Casset e Melhora Contínua
    casset_bonus_qualidade = 0.5
    casset_bonus_satisfacao = 0.3
    casset_reducao_tempo = 0.8  # Agora usado com astype(float)
    casset_incremento_qualidade = 0.1
    casset_incremento_satisfacao = 0.05

    for i, row in df.iterrows():
        if row['UsaCasset']:
            # Efeito base (agora com operações em float)
            df.loc[i, 'QualidadeProduto'] += casset_bonus_qualidade
            df.loc[i, 'SatisfacaoParticipantes'] += casset_bonus_satisfacao
            df.loc[i, 'TempoEtapa1'] = df.loc[i, 'TempoEtapa1'] * casset_reducao_tempo
            df.loc[i, 'TempoEtapa2'] = df.loc[i, 'TempoEtapa2'] * casset_reducao_tempo
            df.loc[i, 'TempoEtapa3'] = df.loc[i, 'TempoEtapa3'] * casset_reducao_tempo

            # Efeito de melhora contínua
            projetos_casset_anteriores = df.loc[:i-1, 'UsaCasset'].sum()
            df.loc[i, 'QualidadeProduto'] += projetos_casset_anteriores * casset_incremento_qualidade
            df.loc[i, 'SatisfacaoParticipantes'] += projetos_casset_anteriores * casset_incremento_satisfacao

            # Centralidade da comunicação (moderada)
            df.loc[i, 'CentralidadeComunicacao'] = np.clip(np.random.normal(0.65, 0.1), 0.4, 0.8)
            #Interacoes
            df.loc[i, 'InteracoesDocs'] = df.loc[i, 'InteracoesDocs'] * 1.2
            df.loc[i, 'InteracoesEmail'] = df.loc[i, 'InteracoesEmail'] * 1.1
            df.loc[i, 'InteracoesReunioes'] = df.loc[i, 'InteracoesReunioes'] * 1.1
        else:
            # Centralidade influenciada pela complexidade (sem Casset)
            df.loc[i, 'CentralidadeComunicacao'] = np.clip(0.3 + (row['ComplexidadeTarefa'] / 10) * 0.4, 0.2, 0.9)

        # Influência da diversidade, feedback, liderança e complexidade
        df.loc[i, 'QualidadeProduto'] += (row['DiversidadeEquipe'] * 0.5) + (row['FrequenciaFeedback'] * 0.2) + (row['LiderancaDistribuida'] * 0.4) - ((10 - row['ComplexidadeTarefa']) * 0.1)
        df.loc[i, 'SatisfacaoParticipantes'] += (row['DiversidadeEquipe'] * 0.3) + (row['FrequenciaFeedback'] * 0.1) + (row['LiderancaDistribuida'] * 0.6)

    # Ajustes finais (clipping) e arredondamento
    df['TempoEtapa1'] = df['TempoEtapa1'].clip(lower=1).round(0).astype(int) #Arredonda e converte
    df['TempoEtapa2'] = df['TempoEtapa2'].clip(lower=1).round(0).astype(int) #Arredonda e converte
    df['TempoEtapa3'] = df['TempoEtapa3'].clip(lower=1).round(0).astype(int) #Arredonda e converte
    df['QualidadeProduto'] = df['QualidadeProduto'].clip(upper=10).round(1)
    df['SatisfacaoParticipantes'] = df['SatisfacaoParticipantes'].clip(upper=10).round(1)
    df['CentralidadeComunicacao'] = df['CentralidadeComunicacao'].round(2)
    df['InteracoesDocs'] = df['InteracoesDocs'].round(0).astype(int)  # Arredonda e converte
    df['InteracoesEmail'] = df['InteracoesEmail'].round(0).astype(int) # Arredonda e converte
    df['InteracoesReunioes'] = df['InteracoesReunioes'].round(0).astype(int) # Arredonda e converte

    return df

# --- Funções de Visualização ---

def generate_boxplot(df, x, y, title="", filename="", hue=None):
    create_figure()
    if hue is not None:
        sns.boxplot(x=x, y=y, data=df, palette=DEFAULT_PALETTE, hue=hue)
    else:
        sns.boxplot(x=x, y=y, data=df, palette=DEFAULT_PALETTE)
    plt.title(title)
    plt.xticks(rotation=45, ha="right")
    save_fig(filename)

def generate_scatterplot(df, x, y, title="", filename="", hue=None, style=None):
    create_figure()
    sns.scatterplot(x=x, y=y, data=df, hue=hue, style=style, palette=DEFAULT_PALETTE, alpha=0.8)
    plt.title(title)
    save_fig(filename)

def generate_histogram(df, column, title="", filename="", bins=20, kde=True):
    create_figure()
    sns.histplot(data=df, x=column, kde=kde, bins=bins, color='skyblue')
    plt.title(title)
    save_fig(filename)

def generate_network_graph(df, title="", filename=""):
    G = nx.Graph()
    for i, row in df.iterrows():
        for j in range(i + 1, len(df)):
            G.add_edge(i, j, weight=row['InteracoesDocs'] + row['InteracoesEmail'] + row['InteracoesReunioes'])

    create_figure()
    pos = nx.spring_layout(G, seed=42)
    weights = [G[u][v]['weight'] / 10 for u, v in G.edges()]
    nx.draw(G, pos, with_labels=True, node_color='cyan', edge_color='gray', width=weights, node_size=800)
    plt.title(title)
    save_fig(filename)

def generate_heatmap(df, title="", filename="", annot=True):
    create_figure()
    corr = df.corr()
    sns.heatmap(corr, annot=annot, cmap=DEFAULT_PALETTE, fmt=".2f", linewidths=.5)
    plt.title(title)
    save_fig(filename)

# --- Funções de Análise Estatística ---

def perform_t_test(df, group_col, value_col, print_results=True):
    group1 = df[df[group_col] == True][value_col].dropna()
    group2 = df[df[group_col] == False][value_col].dropna()
    if group1.empty or group2.empty:
        print(f"Não há dados suficientes para o teste t em {value_col}")
        return None, None
    t_stat, p_value = stats.ttest_ind(group1, group2, equal_var=False)
    if print_results:
        print(f"Teste t ({value_col} - Casset vs. Não Casset): t={t_stat:.3f}, p={p_value:.3f}")
    return t_stat, p_value

def perform_linear_regression(df, x_cols, y_col, print_results=True):
    if y_col not in df.columns:
        print(f"Coluna '{y_col}' não encontrada.")
        return None
    for col in x_cols:
        if col not in df.columns:
            print(f"Coluna '{col}' não encontrada em x_cols.")
            return None

    X = df[x_cols].dropna()
    y = df[y_col].dropna()
    y = y.loc[X.index]

    if X.empty or y.empty:
        print("Não há dados suficientes para a regressão após a limpeza.")
        return None

    X = sm.add_constant(X)
    try:
        model = sm.OLS(y, X).fit()
        if print_results:
            print(model.summary())
        return model
    except ValueError as e:
        print(f"Erro na regressão: {e}")
        return None
    except Exception as e:
        print(f"Erro inesperado: {e}")
        return None

# --- Função Principal (main) ---
if __name__ == "__main__":
    drive.mount(DRIVE_MOUNT_PATH, force_remount=True)
    os.makedirs(GRAPHPATH, exist_ok=True)
    os.makedirs(SUMMARYPATH, exist_ok=True)

    # --- Gerar Dados Sintéticos (Nova Heurística) ---
    df = generate_synthetic_data_improved()

    # --- Análises e Visualizações ---

    df['TempoTotal'] = df['TempoEtapa1'] + df['TempoEtapa2'] + df['TempoEtapa3']
    generate_boxplot(df, 'UsaCasset', 'TempoTotal',
                     title="Tempo Total do Projeto (Casset vs. Não Casset)",
                     filename="1_boxplot_tempo_total.png")

    generate_boxplot(df, 'UsaCasset', 'QualidadeProduto',
                     title="Qualidade do Produto Final (Casset vs. Não Casset)",
                     filename="2_boxplot_qualidade.png")

    generate_boxplot(df, 'UsaCasset', 'SatisfacaoParticipantes',
                    title="Satisfação dos Participantes (Casset vs. Não Casset)",
                    filename="3_boxplot_satisfacao.png")

    df['InteracoesTotais'] = df['InteracoesDocs'] + df['InteracoesEmail'] + df['InteracoesReunioes']

    # Troca de scatterplot por gráfico de barras para InteracoesTotais vs. Qualidade
    create_figure()
    sns.barplot(x='UsaCasset', y='QualidadeProduto', data=df, hue='UsaCasset', palette=DEFAULT_PALETTE, errorbar=None)
    plt.title("Média da Qualidade do Produto por Uso do Casset")
    plt.xlabel("Usa Casset")
    plt.ylabel("Qualidade Média do Produto")
    save_fig("4_barplot_interacoes_qualidade.png")


    generate_scatterplot(df, 'CentralidadeComunicacao', 'TempoTotal',
                         title="Centralidade da Comunicação vs. Tempo Total",
                         filename="5_scatter_centralidade_tempo.png", hue='UsaCasset')

    generate_histogram(df, 'QualidadeProduto',
                       title="Distribuição da Qualidade do Produto Final",
                       filename="6_hist_qualidade.png")

    generate_histogram(df, 'SatisfacaoParticipantes',
                       title="Distribuição da Satisfação dos Participantes",
                       filename="7_hist_satisfacao.png")

    generate_network_graph(df.sample(10),
                           title="Rede de Colaboração (Amostra)",
                           filename="8_network_graph.png")

    generate_heatmap(df, title="Heatmap de Correlação (Variáveis Derivadas)",
                     filename="9_heatmap.png")

    features = ['UsaCasset', 'InteracoesTotais', 'CentralidadeComunicacao', 'NumParticipantes', 'DiversidadeEquipe', 'FrequenciaFeedback', 'LiderancaDistribuida', 'ComplexidadeTarefa']
    perform_linear_regression(df, features, 'QualidadeProduto', print_results=True)

    perform_t_test(df, 'UsaCasset', 'TempoTotal')
    perform_t_test(df, 'UsaCasset', 'QualidadeProduto')
    perform_t_test(df, 'UsaCasset', 'SatisfacaoParticipantes')

    df_numeric = df.select_dtypes(include=np.number).dropna()
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(df_numeric)
    pca = PCA(n_components=2)
    principal_components = pca.fit_transform(scaled_data)
    pca_df = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2'])
    pca_df['UsaCasset'] = df['UsaCasset']

    create_figure()
    sns.scatterplot(x='PC1', y='PC2', data=pca_df, hue='UsaCasset', palette=DEFAULT_PALETTE)
    plt.title("PCA - Componentes Principais (Colorido por Uso do Casset)")
    save_fig("12_pca_plot.png")

    generate_scatterplot(df, 'ProjetoID', 'QualidadeProduto',
                        title="Qualidade do Produto ao Longo do Tempo",
                        filename="13_scatter_qualidade_tempo.png", hue='UsaCasset')

    print("Análise Casset Completa (Nova Heurística)!")

Mounted at /content/drive


<ipython-input-6-9a8c86a11330>:99: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '5.6000000000000005' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, 'TempoEtapa1'] = df.loc[i, 'TempoEtapa1'] * casset_reducao_tempo
<ipython-input-6-9a8c86a11330>:100: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '14.4' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, 'TempoEtapa2'] = df.loc[i, 'TempoEtapa2'] * casset_reducao_tempo
<ipython-input-6-9a8c86a11330>:101: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2.4000000000000004' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, 'TempoEtapa3'] = df.loc[i, 'Tempo

Erro na regressão: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).
Teste t (TempoTotal - Casset vs. Não Casset): t=-5.289, p=0.000
Teste t (QualidadeProduto - Casset vs. Não Casset): t=2.444, p=0.018
Teste t (SatisfacaoParticipantes - Casset vs. Não Casset): t=2.623, p=0.012
Análise Casset Completa (Nova Heurística)!
